# Problem 1: Logging
We need to write a log file that saves each of the following variables. Create the string for the log file. Include the data and time at the top of the log file.

In [ ]:
voltage_scale = 0.1 # V / div
time_scale = 1e-3 # s / div
voltage_offset = -0.5 # V
time_offset = 0 # s
notes = 'Connected in loopback with mini BNC'

log = '# -------- File I/O project log file -------- #\n'
# Your code here
from datetime import datetime
dt = datetime.now().strftime("%d-%m-%Y,%H:%M:%S.%f")
log += 'Time: ' + dt + '\n'
log += f'Voltage scale: {voltage_scale} V/div\n'
log += f'Voltage offset: {voltage_offset} V\n'
log += f'Time scale: {time_scale} s/div\n'
log += f'Time offset: {time_offset} s\n'
log += f'Notes: {notes}\n'

Now, save the file in the current directory to 'fileio.log'. <b>Bonus</b>: raise an error if the file already exists.

In [ ]:
import os 
if os.path.exists('fileio.log'):
    raise FileExistsError('fileio.log already exists')
with open('fileio.log', 'w') as file:
    file.write(log)

Write a function that takes the path to the log file, imports the file, and returns the voltage, temperature, notes, and datetime. Your code should return the voltage in V and the temperature in K.

In [ ]:
def read_log(path):
    """
    Read the log file and return the contents.

    :param path: str, path to the log file 

    :return voltage: float, voltage in V 
    :return temperature: float, temperature in K 
    :return notes: str, measurement setup notes 
    :return dt: datetime.datetime, datetime of the measurement
    """
    # Your code here 
    with open(path, 'r') as file:
        lines = file.readlines()
    dt = [line for line in lines if line.startswith('Time: ')][0]
    dt = dt.replace('Time: ', '').replace('\n', '')
    dt = datetime.strptime(dt, "%d-%m-%Y,%H:%M:%S.%f")
    voltage_scale = [line for line in lines if line.startswith('Voltage scale: ')][0]
    voltage_scale = voltage_scale.replace('Voltage scale: ', '').replace('V/div\n', '')
    voltage_scale = float(voltage_scale)
    voltage_offset = [line for line in lines if line.startswith('Voltage offset: ')][0]
    voltage_offset = voltage_offset.replace('Voltage offset: ', '').replace('V\n', '')
    voltage_offset = float(voltage_offset)
    time_scale = [line for line in lines if line.startswith('Time scale: ')][0]
    time_scale = time_scale.replace('Time scale: ', '').replace('s/div\n', '')
    time_scale = float(time_scale)
    time_offset = [line for line in lines if line.startswith('Time offset: ')][0]
    time_offset = time_offset.replace('Time offset: ', '').replace('s\n', '')
    time_offset = float(time_offset)
    notes = [line for line in lines if line.startswith('Notes: ')][0]
    notes = notes.replace('Notes: ', '').replace('\n', '')
    return voltage_scale, voltage_offset, time_scale, time_offset, notes, dt

voltage_scale, voltage_offset, time_scale, time_offset, notes, dt =\
    read_log('fileio.log')

# Problem 2: Instrument control
We will connect to a fake instrument and take data, then FFT the data to extract a signal. First, we will import `fake_pyvisa`, which will act like `pyvisa` without needing to connect to a physical instrument. We will also import `numpy`, `matplotlib.pyplot`, and our simple fft function.

In [ ]:
import fake_pyvisa
import numpy as np 
import matplotlib.pyplot as plt
from simple_fft import simple_fft

Create the resource manager instance. We will connect to the instrument via LAN. The IP address is '192.168.2.125'. Create the instrument instance and query the device ID to confirm the connection.

In [ ]:
rm = fake_pyvisa.ResourceManager()

In [ ]:
ip_address = '192.168.2.125'
inst = rm.open_resource(f'TCPIP0::{ip_address}::INSTR') 
inst.query('*IDN?')

We expect our signal to be have a range between 1 V and 10 V, so we need to set the voltage range to 10 V, rather than the default 1 V. From the programming manual, we find that the command syntax is 'C[channel]:VOLT_DIV [range]V', where channel is the channel index and range is the voltage range. Set the voltage range on the first channel to 10 V.

In [ ]:
inst.write('C1:VOLT_DIV 10V')

From the programming manual, we find that the command to get the sample rate is 'SARA?'. Query the sample rate, and create the variable 'tsample' in s.

In [ ]:
response = inst.query('SARA?')
tsample = float(response.replace('SARA ', '').replace('Sa/s\n', ''))

From the programming manual, we find that the command to recieve data is 'C[channel]:WF? DAT2'. Query the device for data and create the variable 'x'. Create the corresponding time array 'time'.

In [ ]:
x = inst.query('C1:WF? DAT2')
time = np.arange(0, len(x) * tsample, tsample)

Plot the signal timestream. Don't forget the axis labels.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [12, 4])
ax.set(xlabel = 'time (s)', ylabel = 'voltage (V)')
ax.plot(time, x)

FFT the data and plot the result. Can you extract the signal? If not, try taking data again. The noise in this system is on the edge of our ability to extract the peak, so we may find that the peak is visible in some datasets and not others. You may find that you cannot extract the signal out of the noise at all: we will need averaging.

In [ ]:
f, y = simple_fft(tsample, x)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [12, 4])
ax.set(xlabel = 'Frequency (Hz)', ylabel = 'voltage (mV)')
ax.plot(f, y * 1e3)

ix = np.argmax(y)
ax.plot(f[ix], y[ix] * 1e3, marker = 'x', color = 'black')
print(f'Signal frequency: {f[ix]} Hz')
print(f'Signal amplitude: {round(1e3 * y[ix], 2)} mV')

### Problem 2.2: Averaging
The data above is fairly noisy. Take many sets of data, and average the FFTs to reduce the noise. How many sets of data do you need before the noise looks acceptable to you?

In [ ]:
ys = []
for i in range(1000):
    x = inst.query('C1:WF? DAT2')
    f, y = simple_fft(tsample, x)
    ys.append(y)
y_avg = np.mean(ys, axis = 0)
f, y_avg = f[1:], y_avg[1:] 
# Sometimes the DC component (f = 0) can offset the plot, so we can remove it

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [12, 4])
ax.set(xlabel = 'Frequency (Hz)', ylabel = 'voltage (mV)')
ax.plot(f, y_avg * 1e3)

ix = np.argmax(y_avg)
ax.plot(f[ix], y_avg[ix] * 1e3, marker = 'x', color = 'black')
print(f'Signal frequency: {f[ix]} Hz')
print(f'Signal amplitude: {round(1e3 * y_avg[ix], 2)} mV')

### Note
In practice, it is more common to average the square of the voltage (power), which will give us higher signal-to-noise. We have kept things simple for this lab by averaging voltage, but you can mess around with the square to see how it performs.

# Problem 3: Saving csv files
Format and save the following data as a single column csv file with column name `voltage`. Some people prefer to include the units as the second row in the columns, and others prefer to save the units in the log file. You can make this choice for yourself here. The units for the list you are given are mV. Turn this code into a function, which takes the following parameters as inputs: `voltage`, `path`, and optionally `unit`. 
**Note:** If you open a csv file in excel, it will try to get you to save it as an xlsx file. This is a completely different file format, so make sure to keep the file format as csv. 

In [ ]:
voltage = [-22.49194569,  -8.84048421, -24.08512566, -28.17491177,
            17.24362829, -31.72005968, -15.03070767,  -5.74553134,
            -3.6421384 , -10.66114773, -16.61791335,   0.62631091,
           -31.21654304,   1.64264594, -17.329834  , -13.47011889,
           -19.19854741,  -5.61232529,  -2.21477568, -23.97114704,
            -1.01685989,  -3.64775334,  -1.60606257,  -0.67603204,
            -5.94722628, -19.91495162,  -4.08330075,   3.25255982,
            -0.16814809,  -5.22093793,  -7.09634211, -16.28708169,
           -16.03094174,  -9.89704766,  -4.3526658 ,   8.84505229,
           -10.10431137, -21.98260157,  21.97641644,  -5.87630593] # mV

In [ ]:
import os
def save_voltage(voltage, path, unit):
    """
    Save the voltage array to a csv file with units as the header 

    :param voltage: array-like, voltage array 
    :param path: str, path to save the data 
    :param unit: str, units of the voltage array 
    """
    # Your code here 
    if os.path.exists(path):
        raise FileExistsError(f'{path} already exists')
    output = f'voltage\n{unit}\n' 
    for v in voltage:
        output += f'{v}\n' 
    with open(path, 'w') as file:
        file.write(output)
        
save_voltage(voltage, 'pset3_voltage.csv', 'mV')

Write a function that reads the data from the csv file. The function should take the parameter `path` and return an array of voltages. Your function should work for the following units: 'uV', 'mV', 'V', and 'kV'.

In [ ]:
def read_voltage(path):
    """ 
    Reads the voltage array from the csv file

    :param path: str, path to the voltage data 

    :return voltage: list, voltages in V 
    """
    # Your code here
    multipliers = {'uV': 1e-6, 'mV': 1e-3, '': 1, 'kV': 1e3} 
    with open(path, 'r') as file:
        lines = file.readlines()
    lines = [line.replace('\n', '') for line in lines]
    unit = lines[1]  
    m = multipliers[unit]
    voltage = [float(line) * m for line in lines[2:]]
    return voltage
    
read_voltage('pset3_voltage.csv')

## Problem 3.2 Bonus 
Repeat the tasks above using the `pandas` package. 

In [ ]:
import pandas as pd 

In [ ]:
def save_voltage(voltage, path, unit):
    """
    Save the voltage array to a csv file with units as the header 

    :param voltage: array-like, voltage array 
    :param path: str, path to save the data 
    :param unit: str, units of the voltage array 
    """
    # Your code here 
    header = pd.DataFrame({'voltage': [unit]})
    df = pd.DataFrame({'voltage': voltage}) 
    df = pd.concat([header, df]).reset_index(drop = True)
    df.to_csv(path, index = False)
    
save_voltage(voltage, 'pset3_voltage.csv', 'mV')

In [ ]:
def read_voltage(path):
    """ 
    Reads the voltage array from the csv file

    :param path: str, path to the voltage data 

    :return voltage: list, voltages in V 
    """
    # Your code here
    multipliers = {'uV': 1e-6, 'mV': 1e-3, '': 1, 'kV': 1e3} 
    df = pd.read_csv(path, skiprows = [1])
    units = pd.read_csv(path, nrows = 1) 
    unit = units['voltage'].loc[0] 
    m = multipliers[unit] 
    voltage = np.array(df.voltage) * m
    return voltage
    
read_voltage('pset3_voltage.csv')

This may seem complicated, but most of the work comes from the way we have decided to handle units. If we instead standardize our output files to use V, we can ignore the units and the problem becomes simple with pandas.`

In [ ]:
import pandas as pd 

In [ ]:
def save_voltage(voltage, path):
    """
    Save the voltage array to a csv file

    :param voltage: array-like, voltage array in V
    :param path: str, path to save the data 
    """
    df = pd.DataFrame({'voltage': voltage}) 
    df.to_csv(path, index = False)
    
save_voltage(np.array(voltage) * 1e-3, 'pset3_voltage.csv')

In [ ]:
def read_voltage(path):
    """ 
    Reads the voltage array from the csv file

    :param path: str, path to the voltage data 

    :return voltage: np.array, voltages in V 
    """
    df = pd.read_csv(path)
    voltage = np.array(df.voltage)
    return voltage
    
read_voltage('pset3_voltage.csv')